# Investigating the Multiprogramming Ability of EC2 Instances using a Cost-Benefit Analysis

<strong> Team 001 </strong>: Jason Platt, Jennifer Chien, Shuhao Chang, Yuanzhen Lin, Srishti Palani

<strong> Question </strong>: How can we get the best multiprogramming ability from EC2 instances with a fixed budget? 

Introduction: 

- Cloud Computing enhances allows us to run computations on remote computer system resources, when we want to, without having to worry about actively managing resources. However, this ability comes at a price. Cloud providers typically use a "pay-as-you-go" model, which can lead to unexpected operating expenses if programmers or administrators are not familiar with cloud-pricing models. Since we want to maximize CPU time and utility for the cost of using an EC2 instance, in this project, we investigate the multiprogramming ability of different EC2 instances using a cost-benefit analysis.

- Multi-programming Ability: 
    - One of the most important roles of an Operating System is to assign compute power to the many processes that are need to be executed. 
    - In a non-multiprogrammed system, as soon as one job leaves the CPU and goes for some other task (say I/O ), the CPU becomes idle. The CPU keeps waiting and waiting until this job (which was executing earlier) comes back and resumes its execution with the CPU. So CPU remains free for all this while. 
    - Now it has a drawback that the CPU remains idle for a very long period of time. Also, other jobs which are waiting to be executed might not get a chance to execute because the CPU is still allocated to the earlier job.
    - This poses a very serious problem that even though other jobs are ready to execute, CPU is not allocated to them as the CPU is allocated to a job which is not even utilizing it (as it is busy in I/O tasks).
    - It cannot happen that one job is using the CPU for say 1 hour while the others have been waiting in the queue for 5 hours. To avoid situations like this and come up with efficient utilization of CPU, the concept of multi programming came up.
    - The main idea of multi programming is to maximize the CPU time.
    - Multiprogramming ability can be measured by the number of processes when CPU utility reaches the maximum point. When the number of processes does not exceed this point, the CPU is not fully used. After this point, cache and memory are not large enough to support all processes and they have to frequently require disk I/O, which leads to the decrease of CPU utilization called "Thrashing". 
    - Due to various cache and memory configuration of different types of EC2 instances, they have different multiprogramming levels. We want to check the maximum multiprogramming ability of different AWS EC2 instances.

- Method: To investigate the multiprogramming ability of different EC2 instances at different costs we: 
    - (i) wrote a standard python test script that  consumes a large size of memory and CPU time. We chose to multiply a NXN matrix that consumed approximately 1GB of memory
    - (ii) wrote a driver script which keeps forking new processes with the standard test program and measure the CPU utility at the same time. 
    - (iii) compared the spot prices of EC2 instances and their multiprogramming levels
    - (iv) used python libraries to parallel matrix multiplication, to see how speedup is related to increased threads.
    - (v) run the scripts on 12 different EC2 instances chosen based on their memory capacity, number of cores, and cost. 
    - (vi) analyze the differences to conclude which one is attractive at a reasonable price. 

  




#  Data

In [13]:
import pandas as pd

print(pd.__version__)

df=pd.read_csv('/Users/srishtipalani/Downloads/All_Data.csv')
df.head(5)

1.0.1


,c5large_size_arr,c5large_num_proc,c5large_time_avg,c5large_time_std,c5large_cpu_ut_avg,c5large_cpu_ut_std,c5xlarge_size_arr,c5xlarge_num_proc,c5xlarge_time_avg,c5xlarge_time_std,...,m52xlarge_time_avg,m52xlarge_time_std,m52xlarge_cpu_ut_avg,m52xlarge_cpu_ut_std,m54xlarge_size_arr,m54xlarge_num_proc,m54xlarge_time_avg,m54xlarge_time_std,m54xlarge_cpu_ut_avg,m54xlarge_cpu_ut_std
0,1.0,1.0,0.001975,0.000489,7.500,23.848480,1.0,1.0,0.002566,0.000620,...,0.002366,0.000362,6.2500,24.20614591,1,1,0.002384,0.000356,0.93750,8.788975694
1,1.0,2.0,0.002731,0.000074,25.000,43.301270,1.0,2.0,0.002808,0.000173,...,0.002817,0.000082,11.2500,31.59806165,1,2,0.002892,0.000042,4.06250,19.34221016
2,1.0,4.0,0.005025,0.000040,47.500,48.669806,1.0,4.0,0.004706,0.000191,...,0.004601,0.000164,15.6250,34.08972536,1,4,0.004354,0.000049,4.06250,19.34221016
3,1.0,8.0,0.010276,0.001023,83.335,35.745241,1.0,8.0,0.008722,0.000605,...,0.008332,0.000526,33.7500,44.56385867,1,8,0.007967,0.000433,12.18750,30.48917585
4,1.0,16.0,0.020079,0.000243,98.335,7.257567,1.0,16.0,0.016030,0.000468,...,0.015340,0.000236,38.4375,37.58115676,1,16,0.014663,0.000111,14.26875,27.50151629


In [14]:
df=pd.read_csv('/Users/srishtipalani/Downloads/c5large.csv')
df.head(5)

,size_arr,num_proc,time_avg,time_std,cpu_ut_avg,cpu_ut_std
0,1,1,0.001975,0.000489,7.500,23.848480
1,1,2,0.002731,0.000074,25.000,43.301270
2,1,4,0.005025,0.000040,47.500,48.669806
3,1,8,0.010276,0.001023,83.335,35.745241
4,1,16,0.020079,0.000243,98.335,7.257567


In [15]:
df=pd.read_csv('/Users/srishtipalani/Downloads/c5xlarge.csv')
df.head(5)

,size_arr,num_proc,time_avg,time_std,cpu_ut_avg,cpu_ut_std
0,1,1,0.002566,0.000620,8.750,27.128168
1,1,2,0.002808,0.000173,10.000,30.000000
2,1,4,0.004706,0.000191,21.250,38.547860
3,1,8,0.008722,0.000605,42.500,45.483513
4,1,16,0.016030,0.000468,67.085,32.164379


In [16]:
df=pd.read_csv('/Users/srishtipalani/Downloads/c52xlarge.csv')
df.head(5)

,size_arr,num_proc,time_avg,time_std,cpu_ut_avg,cpu_ut_std
0,1,1,0.002417,0.000372,1.87500,12.358575
1,1,2,0.002833,0.000121,5.00000,21.794495
2,1,4,0.004619,0.000213,9.37500,27.492897
3,1,8,0.008368,0.000667,16.87500,33.488571
4,1,16,0.015737,0.000165,27.29125,34.686998


In [18]:
df=pd.read_csv('/Users/srishtipalani/Downloads/m5large.csv')
df.head(5)

,size_arr,num_proc,time_avg,time_std,cpu_ut_avg,cpu_ut_std
0,1,1,0.003021,0.000764,30.00,45.825757
1,1,2,0.004227,0.000770,45.00,49.749372
2,1,4,0.008209,0.001831,65.00,47.696960
3,1,8,0.017052,0.004070,95.00,21.794495
4,1,16,0.031873,0.002785,97.75,6.796139


In [19]:
df=pd.read_csv('/Users/srishtipalani/Downloads/m5xlarge.csv')
df.head(5)

,size_arr,num_proc,time_avg,time_std,cpu_ut_avg,cpu_ut_std
0,1,1,0.002229,0.000374,5.000,21.794495
1,1,2,0.002713,0.000139,15.000,35.707142
2,1,4,0.004986,0.000374,36.250,47.417692
3,1,8,0.009233,0.001129,66.250,45.259667
4,1,16,0.016541,0.000508,93.335,16.156184


In [21]:
df=pd.read_csv('/Users/srishtipalani/Downloads/m52xlarge.csv')
df.head(5)

,size_arr,num_proc,time_avg,time_std,cpu_ut_avg,cpu_ut_std
0,1,1,0.002366,0.000362,6.2500,24.206146
1,1,2,0.002817,0.000082,11.2500,31.598062
2,1,4,0.004601,0.000164,15.6250,34.089725
3,1,8,0.008332,0.000526,33.7500,44.563859
4,1,16,0.015340,0.000236,38.4375,37.581157


In [22]:
df=pd.read_csv('/Users/srishtipalani/Downloads/m54xlarge.csv')
df.head(5)

,size_arr,num_proc,time_avg,time_std,cpu_ut_avg,cpu_ut_std
0,1,1,0.002384,0.000356,0.93750,8.788976
1,1,2,0.002892,0.000042,4.06250,19.342210
2,1,4,0.004354,0.000049,4.06250,19.342210
3,1,8,0.007967,0.000433,12.18750,30.489176
4,1,16,0.014663,0.000111,14.26875,27.501516


In [23]:
df=pd.read_csv('/Users/srishtipalani/Downloads/r5large.csv')
df.head(5)

,size_arr,num_proc,time_avg,time_std,cpu_ut_avg,cpu_ut_std
0,1,1,0.002109,0.000490,12.500,31.124749
1,1,2,0.002940,0.000059,25.000,43.301270
2,1,4,0.005413,0.000047,38.335,47.464432
3,1,8,0.011106,0.000993,93.335,22.605802
4,1,16,0.022064,0.000386,96.670,9.990000


In [ ]:
df=pd.read_csv('/Users/srishtipalani/Downloads/m52xlarge.csv')
df.head(5)